In [40]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score


# Load the dataset
df1 = pd.read_csv(r"../datasets/weather_classification_data.csv")
# Preprocess the data
df = pd.read_csv(r"../datasets/processed_weather_data.csv")

### Machine learning 

* ML model selection
    - At this point of the project, I'll start the process to perform the __Logistics Regression__ to practice and consolidate the knowledge acquired during the classes (2025/06/16).

In [41]:
df.head()

,num__Temperature,num__Humidity,num__Wind Speed,num__Precipitation (%),num__Atmospheric Pressure,num__UV Index,num__Visibility (km),cat__Cloud Cover_cloudy,cat__Cloud Cover_overcast,cat__Cloud Cover_partly cloudy,cat__Season_Spring,cat__Season_Summer,cat__Season_Winter,cat__Location_inland,cat__Location_mountain
0,-0.294931,0.212404,-0.048086,0.887629,0.134203,-0.520104,-0.582231,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.143035,1.351385,-0.192836,0.543291,0.150602,0.776424,1.345768,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,0.625367,-0.233285,-0.409962,-1.178401,0.346579,0.257813,0.010999,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.085516,0.707613,-1.206089,0.887629,0.549008,0.776424,-1.323769,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.452811,0.261924,1.037543,0.386773,-0.407490,-0.779410,-0.878846,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [42]:
df1.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [43]:
# Creating binary columns for Weather Dryness - 1 for dry weather, 0 for not dry
# Assuming 'precipitation', 'humidity', and 'temperature' as the defining features for dryness
df1['Dryness'] = np.where((df1["Precipitation (%)"] < 10) & (df1["Humidity"] < 50) & (df1["Temperature"] > 25), 1, 0)

In [44]:
df1 = df1.drop(columns=['Weather Type'])

In [45]:
df1.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Dryness
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,0
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,0
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,0
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,0
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,0


In [46]:
# # Saving the modified DataFrame to a new CSV file
# df1.to_csv(r"../datasets/weather_data_target.csv", index=False)

In [47]:
X = df

y = (df1['Dryness'])

In [48]:
y.shape

(13200,)

In [49]:
y.value_counts()

Dryness
0    12618
1      582
Name: count, dtype: int64

1 for Dry, 0 for Not Dry

In [50]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Dryness, dtype: int64

### Logistical Regression

In [51]:
smote = SMOTE(random_state=42)
X_upsampled, y_upsampled = smote.fit_resample(X, y)

In [52]:
y_upsampled.value_counts()

Dryness
0    12618
1    12618
Name: count, dtype: int64

In [53]:
X_train_up, X_test_up, y_train_up, y_test_up = train_test_split(X_upsampled, y_upsampled, test_size=0.2, random_state=42)

X_train_up.shape, X_test_up.shape, y_train_up.shape, y_test_up.shape

((20188, 15), (5048, 15), (20188,), (5048,))

__UP SAMPLING TRAINING DATAPOINTS__

In [54]:
lr = LogisticRegression(random_state=16, max_iter=1000)
lr.fit(X_train_up, y_train_up)

LogisticRegression(max_iter=1000, random_state=16)

In [55]:
y_pred = lr.predict(X_test_up)
y_pred

array([0, 1, 1, ..., 0, 1, 1], shape=(5048,))

In [56]:
diff_df = pd.DataFrame({'Actual' : y_test_up, 'Predicted': y_pred})
diff_df

,Actual,Predicted
7712,0,0
18700,1,1
13924,1,1
14205,1,1
20042,1,1
...,...,...
19784,1,1
19912,1,1
102,0,0
21603,1,1


In [57]:
conf_matrix = confusion_matrix(y_test_up, y_pred)
conf_matrix

array([[2435,  123],
       [  32, 2458]])

### Observations:
#### True Negatives (TN) = 2404
The model correctly predicted 2520 the weather as "Not Dry".
#### False Positives (FP) = 130
The model incorrectly predicted 130 the weather as "Dry". When those actually didn't (Type I error).
#### False Negatives (FN) = 1
The model incorrectly predicted 1 the weather as "Not Dry". When they actually was dry (Type II error).
#### True Positives (TP) = 105
The model correctly predicted 105 the weather as "Dry".

In [58]:
target_names = ['Not dry weather', 'dry weather']
print(classification_report(y_test_up, y_pred, target_names=target_names))

                 precision    recall  f1-score   support

Not dry weather       0.99      0.95      0.97      2558
    dry weather       0.95      0.99      0.97      2490

       accuracy                           0.97      5048
      macro avg       0.97      0.97      0.97      5048
   weighted avg       0.97      0.97      0.97      5048



In [59]:
cross_val_scores = cross_val_score(lr, X_train_up, y_train_up, cv=5)
print(f"Cross-validation scores: {cross_val_scores}")

Cross-validation scores: [0.96978702 0.96433878 0.97003467 0.97052267 0.97002725]


### Observations:
* Accuracy = 97% → The model correctly classifies 97% of all instances in the dataset.

* Precision (Positive Predictive Value):
 - "Not Dry weather": 0.99 → When the model predicts "Not Dry", it's correct 99% of the time.
 - "Dry weather": 0.95 → When the model predicts "Dried weather", it's correct 95% of the time.

* Recall (Sensitivity/True Positive Rate):
 - Not Dry weather: 0.95 → The model correctly identifies 95% of all actual "Not dried weathers" cases.
 - Dry weather: 0.99 → The model correctly identifies 99% of all actual "dry weather" cases.

* F1-score (Balance between Precision & Recall):
 - Not Dry: 0.97
 - Dry: 0.97
 - Since the F1-score for "Dry" is lower, the model struggles more with correctly predicting positive cases.

### Key Observations

The logistic regression model demonstrated strong predictive power in identifying dry weather conditions, achieving a high recall rate of 99%, which means it successfully captured nearly all actual dry days.

 * This is a critical factor in reducing wildfire risks.

 * After balancing the dataset before splitting, the model also achieved a precision of __95%__, which means most predicted dry days were correct.

 * This balance between recall and precision makes the model a reliable early-warning tool to __support wildfire prevention__ strategies.

 * This makes the model useful not only for __forecasting__ but also for understanding the __reasons behind__ impactful events, by showing which weather variables contribute most to dry conditions and helping guide preventive actions.

 * The feature weights from logistic regression help explain which factors drive predictions, allowing better understanding of environmental conditions linked to __fire risk__.

### Future Actions for tests and improvements 
To test more possibilities and check for even better results, the next step involves implementing other classification algorithms.

 * __Random Forest__ Machine Learning model can better capture complex patterns in the data through deeper decision paths.

 * It would also be interesting to test decision trees, gradient boosting, or adjust thresholds to see how it impacts precision and recall.

 * Use __cross-validation__ to validate how consistent the model is across different data splits.

 * Explore the feature importance from tree-based models to compare with logistic regression weights and confirm which variables are most related to dryness.

 * Create visual explanations using SHAP or LIME to improve model interpretability and understand individual predictions.

 * Compare predicted dry days with real fire incident data (if available) to support a deeper environmental diagnosis and decision-making.


### Conclusion

This approach supports my learning journey and strengthens my career goal in Data Science, particularly in applying Machine Learning to predict and prevent wildfire risks, contributing to both the prevention and recovery of degraded areas. 

By starting with a simpler model like logistic regression and progressing toward more advanced algorithms such as Random Forest, this notebook represents an important step in moving from foundational techniques to more complex and precise solutions.

These improvements aim not only to enhance precision but also to increase the overall reliability of predictions in real environmental scenarios.